# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
r = requests.get(url)
json_data = r.json()

data_mf = pd.DataFrame(json_data['items'])
data_mf.head(5)

df_rj = data_mf[data_mf['interessado'] == 'Rio de Janeiro']
df_rj.head(5)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,5645,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000161/2013-79,2013-04-08T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,3.031405e+09,1,0,06/05/2013
1,5630,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000158/2013-55,2013-10-22T02:00:00Z,Operação contratual externa (com garantia da U...,Fortalecimento Institucional,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,4.800000e+07,1,0,21/11/2013
2,5635,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000159/2013-08,2013-04-08T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3.135800e+09,1,0,06/05/2013
3,5640,Estado,Rio de Janeiro,33,RJ,None,Arquivado a pedido,17944.000160/2013-24,2013-03-18T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Bank of America Merrill Lynch,Dólar dos EUA,7.164179e+08,0,0,29/07/2013
4,9102,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000903/2009-80,2009-07-29T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco do Brasil S/A,Real,6.110800e+07,1,0,06/08/2009


In [3]:
# 2) Seu código aqui
df_rj['status'].value_counts()

status
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Arquivado por decurso de prazo                            1
Name: count, dtype: int64

In [39]:
# 3) Seu código aqui
ano_status = df_rj['data_status']
df_ano = pd.to_datetime(ano_status, format = '%d/%m/%Y')
df_ano['ano'] = df_ano.dt.year
df_ano = pd.DataFrame(df_ano['ano'])
df_ano.T

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
data_status,2013,2013,2013,2013,2009,2014,2010,2009,2002,2006,...,2016,2012,2011,2016,2009,2012,2011,2008,2009,2016


In [34]:
# 4) Seu código aqui
df_ano['ano'].value_counts()

data_status
2012    11
2009     9
2013     7
2011     6
2014     4
2002     4
2016     4
2010     3
2006     2
2008     2
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [51]:
url2 = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado'
r2 = requests.get(url2)
json_data2 = r2.json()  
df_mg = pd.DataFrame(json_data2['items'])

url3 = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Município'
r3 = requests.get(url3)
json_data3 = r3.json()  
df_mg2 = pd.DataFrame(json_data3['items'])

url4 = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Estado'
r4 = requests.get(url4)
json_data4 = r4.json()
df_ba = pd.DataFrame(json_data4['items'])

url5 = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Município'
r5 = requests.get(url5)
json_data5 = r5.json()
df_ba2 = pd.DataFrame(json_data5['items'])

In [52]:
#1) Seu código aqui
def consulta_pvl(df: pd.DataFrame, uf: str, tipo_interessado: str) -> pd.DataFrame:
       
    if uf not in df['uf'].unique():
        print(f'Estado {uf} não encontrado no DataFrame.')
    
    if tipo_interessado not in ['Estado', 'Município']:
        print('Tipo de interessado inválido. Deve ser "Estado" ou "Município"')

    df_filtrado = df[(df['uf'] == uf) & (df['tipo_interessado'] == tipo_interessado)]

    return df_filtrado

In [53]:
# 2) Seu código aqui
contagem_mg = (df_mg['status'] == 'Arquivado por decurso de prazo').sum()
contagem_mg

1

In [76]:
# 3) Seu código aqui
contagem_ba = consulta_pvl(df_ba2, 'BA', 'Município')
filter = contagem_ba[contagem_ba['status'] == 'Deferido']
filter['interessado'].value_counts()

interessado
Luís Eduardo Magalhães    16
Vitória da Conquista      12
Lauro de Freitas          11
Barreiras                 10
Camaçari                  10
                          ..
Itagibá                    1
São Felipe                 1
Caatiba                    1
Santa Luzia                1
Ribeira do Pombal          1
Name: count, Length: 185, dtype: int64

In [77]:
# 4) Seu código aqui
df_ba.to_csv('DataFrame_Bahia_Estado.csv')